In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_ML_KgapRFE"
outPath = "Results"
foldName = "folds.pickle"

shuffle = True
seed = None

input_data_folder = "PredNTS_MathFeature_ENC"

monitor = 'val_loss'

sub_feature_count = 200

In [2]:
kgap_max = 4

train_data_filename = 'Training-datasets-PredNTS_kgap_{}.csv'
indpe_data_filename = 'independent-dataset-PredNTS_kgap_{}.csv'

In [3]:
import os 
import pickle
import numpy as np
import pandas as pd

# import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report, matthews_corrcoef

from sklearn.feature_selection import RFE

import math

In [4]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [6]:
def get_model(cw = None):
    
    model = RandomForestClassifier(n_estimators=100,
                                   criterion='gini', 
                                   class_weight=cw,
                                   bootstrap=True,
                                   oob_score=True, 
                                  )
    
    return model

# Train data preparation

In [7]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):
    
    current_train_data_filepath = os.path.join(input_data_folder, train_data_filename.format(i))
    current_train_data = pd.read_csv(current_train_data_filepath, sep=',', header=0)
    current_train_data = current_train_data.drop('label', axis=1)
    
    if i == 0:
        train_data = current_train_data
    else:
        train_data = pd.merge(
            train_data,
            current_train_data,
            how="inner",
            on='nameseq'
        )

train_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in train_data['nameseq']])

train_data = train_data.drop('nameseq', axis=1)

train_features = np.array(train_data.drop('label', axis=1))
train_labels = np.array(train_data['label'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

In [8]:
##################################################################################
##### Recursive feature selection
##################################################################################

# model = DecisionTreeClassifier(criterion="gini")
model = get_model()

selector = RFE(model, n_features_to_select=sub_feature_count, step=50)
selector = selector.fit(train_features, train_labels.reshape(train_labels.shape[0]))

feature_indices = np.where(selector.ranking_ == 1)[0]

In [9]:
##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = train_features[:, feature_indices]

folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

input_vec_shape = train_features[0].shape

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Independent data

In [10]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):

    current_indpe_data_filepath = os.path.join(input_data_folder, indpe_data_filename.format(i))
    current_indpe_data = pd.read_csv(current_indpe_data_filepath, sep=',', header=0)
    current_indpe_data = current_indpe_data.drop('label', axis=1)
    
    if i == 0:
        indpe_data = current_indpe_data
    else:
        indpe_data = pd.merge(
            indpe_data,
            current_indpe_data,
            how="inner",
            on='nameseq'
        )

indpe_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in indpe_data['nameseq']])

indpe_data = indpe_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(indpe_data.drop('label', axis=1))
indpe_features = indpe_features[:, feature_indices]

indpe_labels = np.array(indpe_data['label'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

# Training

In [11]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model = get_model()
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    
    model.fit(fold["X_train"][index_arr], fold["y_train"][index_arr].reshape(fold["y_train"].shape[0]))
    
    model_file_obj = open(current_model_path, 'wb')
    pickle.dump(model, model_file_obj)
    model_file_obj.close()
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.

Train/Test model on Fold #1.

Train/Test model on Fold #2.

Train/Test model on Fold #3.

Train/Test model on Fold #4.


## k-fold Training evaluation

In [12]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.803952,0.808272,0.803936,0.796790,0.811083,0.608421
Train,0.996746,0.998319,0.996746,0.995172,0.998320,0.993501


In [13]:
evaluations_df[evaluations_df["Train_Test"] == 'Test']

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.800839,0.790323,"[0.0, 0.8200836820083682, 1.0]","[0.0, 0.2184873949579832, 1.0]","[2, 1, 0]",0.800798,0.820084,0.781513,0.602073
3,1,Test,0.792453,0.803493,"[0.0, 0.773109243697479, 1.0]","[0.0, 0.18828451882845187, 1.0]","[2, 1, 0]",0.792412,0.773109,0.811715,0.585288
5,2,Test,0.846639,0.836735,"[0.0, 0.8613445378151261, 1.0]","[0.0, 0.16806722689075632, 1.0]","[2, 1, 0]",0.846639,0.861345,0.831933,0.693577
7,3,Test,0.771008,0.784141,"[0.0, 0.7478991596638656, 1.0]","[0.0, 0.20588235294117646, 1.0]","[2, 1, 0]",0.771008,0.747899,0.794118,0.542597
9,4,Test,0.808824,0.826667,"[0.0, 0.7815126050420168, 1.0]","[0.0, 0.1638655462184874, 1.0]","[2, 1, 0]",0.808824,0.781513,0.836134,0.618571


In [14]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.997375,1.000000,"[0.0, 0.9947478991596639, 1.0]","[0.0, 0.0, 1.0]","[2, 1, 0]",0.997374,0.994748,1.000000,0.994764
1,0,Test,0.800839,0.790323,"[0.0, 0.8200836820083682, 1.0]","[0.0, 0.2184873949579832, 1.0]","[2, 1, 0]",0.800798,0.820084,0.781513,0.602073
2,1,Train,0.996325,0.996849,"[0.0, 0.9958027282266527, 1.0]","[0.0, 0.0031512605042016808, 1.0]","[2, 1, 0]",0.996326,0.995803,0.996849,0.992651
3,1,Test,0.792453,0.803493,"[0.0, 0.773109243697479, 1.0]","[0.0, 0.18828451882845187, 1.0]","[2, 1, 0]",0.792412,0.773109,0.811715,0.585288
4,2,Train,0.996327,0.997895,"[0.0, 0.9947534102833159, 1.0]","[0.0, 0.002098635886673662, 1.0]","[2, 1, 0]",0.996327,0.994753,0.997901,0.992660
5,2,Test,0.846639,0.836735,"[0.0, 0.8613445378151261, 1.0]","[0.0, 0.16806722689075632, 1.0]","[2, 1, 0]",0.846639,0.861345,0.831933,0.693577
6,3,Train,0.996852,0.996852,"[0.0, 0.9968520461699895, 1.0]","[0.0, 0.0031479538300104933, 1.0]","[2, 1, 0]",0.996852,0.996852,0.996852,0.993704
7,3,Test,0.771008,0.784141,"[0.0, 0.7478991596638656, 1.0]","[0.0, 0.20588235294117646, 1.0]","[2, 1, 0]",0.771008,0.747899,0.794118,0.542597
8,4,Train,0.996852,1.000000,"[0.0, 0.993704092339979, 1.0]","[0.0, 0.0, 1.0]","[2, 1, 0]",0.996852,0.993704,1.000000,0.993724
9,4,Test,0.808824,0.826667,"[0.0, 0.7815126050420168, 1.0]","[0.0, 0.1638655462184874, 1.0]","[2, 1, 0]",0.808824,0.781513,0.836134,0.618571


# Independent data evaluation

## Using k-fold Models

### Performance of each k-fold model

In [15]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model_file_obj = open(current_model_path, 'rb')
    model = pickle.load(model_file_obj)
    model_file_obj.close()

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.640653,0.26292,0.643313,0.647291,0.639335,0.216949


In [16]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.631020,0.251497,"[0.0, 0.6206896551724138, 1.0]","[0.0, 0.3669275929549902, 1.0]","[2, 1, 0]",0.626881,0.620690,0.633072,0.191916
1,1,Independent,0.624490,0.253359,"[0.0, 0.6502463054187192, 1.0]","[0.0, 0.3806262230919765, 1.0]","[2, 1, 0]",0.634810,0.650246,0.619374,0.202778
2,2,Independent,0.640000,0.271154,"[0.0, 0.6945812807881774, 1.0]","[0.0, 0.37084148727984345, 1.0]","[2, 1, 0]",0.661870,0.694581,0.629159,0.243542
3,3,Independent,0.654694,0.271784,"[0.0, 0.645320197044335, 1.0]","[0.0, 0.34344422700587085, 1.0]","[2, 1, 0]",0.650938,0.645320,0.656556,0.229765
4,4,Independent,0.653061,0.266807,"[0.0, 0.625615763546798, 1.0]","[0.0, 0.34148727984344424, 1.0]","[2, 1, 0]",0.642064,0.625616,0.658513,0.216742


### Mean score with k-fold models

In [17]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model_file_obj = open(current_model_path, 'rb')
    model = pickle.load(model_file_obj)
    model_file_obj.close()

    y_pred = model.predict(indpe_features)
    total_pred += y_pred[:, np.newaxis]
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.651429,0.271429,0.686317,0.655172,0.650685,0.23214


### Voting score with k-fold models

In [18]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model_file_obj = open(current_model_path, 'rb')
    model = pickle.load(model_file_obj)
    model_file_obj.close()

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred[:, np.newaxis]
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.651429,0.271429,0.686317,0.655172,0.650685,0.23214


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [19]:
# model = get_model(cw={0:1, 1:0.1})
model = get_model()
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(train_features[index_arr], train_labels[index_arr].reshape(train_labels.shape[0]))

RandomForestClassifier(oob_score=True)

In [20]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Train dataset
##################################################################################

y_pred = model.predict(train_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(train_labels, label_pred)
prec = precision_score(train_labels,label_pred)
mcc = matthews_corrcoef(train_labels, label_pred)

conf = confusion_matrix(train_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(train_labels, y_pred)
auc = roc_auc_score(train_labels, y_pred)

evaluations["Train_Test"].append("Train")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.639184,0.258586,0.635721,0.630542,0.640900,0.205677
Train,0.996222,0.998314,0.996222,0.994123,0.998321,0.992452


In [21]:
print(classification_report(indpe_labels, np.round(y_pred).astype(int)))

              precision    recall  f1-score   support

           0       0.90      0.64      0.75      1022
           1       0.26      0.63      0.37       203

    accuracy                           0.64      1225
   macro avg       0.58      0.64      0.56      1225
weighted avg       0.79      0.64      0.68      1225

